In [65]:
import os, re, subprocess, requests, json, random
from pymediainfo import MediaInfo
from datetime import datetime
import subprocess
from smms import *
from utils import *
import base64, zlib
crf=21.0
qcomp=0.6
aq_mode=3
aq_s=0.6
newline="\r\n"

PROCUCT_DEST_DIR=r'/home/lind/BT'

#https://api.rhilip.info/ptgen.html
DOUBAN_ID="1305327"

SRC="Taipei Story 1985 1080p Carlotta Blu-ray AVC DTS-HD MA 1.0-DIY@PTHome".strip().replace(" ",".")
print("SRC: ",SRC)
REMUX_DIRNAME="Taipei.Story.1985.1080p.Carlotta.Bluray.AVC.Remux.DTS-HD.MA.1.0-PTH"
# if "AVC.Remux" in REMUX_DIRNAME and "DTS-HD.MA" in REMUX_DIRNAME:
#     ENCODE_FILENAME=REMUX_DIRNAME.replace("AVC.Remux","REMUX").replace("DTS-HD.MA.5.1","DTS").replace("DTS-HD.MA.7.1","DTS")+".mkv"
# else:
ENCODE_FILENAME="Taipei.Story.1985.1080p.Carlotta.Bluray.REMUX.DTS-PTH.mkv"

print(REMUX_DIRNAME,"==>",ENCODE_FILENAME)
MOVIE=os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME, REMUX_DIRNAME+".mkv")

ACKNOWLEDGEMENT='''原盘来自ptp@无后缀，字幕取自cmct，感谢。
'''

SOUNDTRACK="【普通话】【简|繁字幕】"
# SOUNDTRACK="【英语】【简|繁|简英|繁英|英字幕】"

SUBTITLE_GZ=1 #官字组

LOG_DIR=os.getcwd()

SRC:  Taipei.Story.1985.1080p.Carlotta.Blu-ray.AVC.DTS-HD.MA.1.0-DIY@PTHome
Taipei.Story.1985.1080p.Carlotta.Bluray.AVC.Remux.DTS-HD.MA.1.0-PTH ==> Taipei.Story.1985.1080p.Carlotta.Bluray.REMUX.DTS-PTH.mkv


In [66]:
cropx=0
cropy=20

x264_filename=ENCODE_FILENAME.replace("REMUX", "x264")
x264_dirname=x264_filename.replace(".mkv","")
x264_fullpath=os.path.join(PROCUCT_DEST_DIR, x264_dirname, x264_filename)
# x265_filename="".join(re.findall(r'([^/\\]*$)', MOVIE)).replace("REMUX", "x265.10bit")
x265_filename=ENCODE_FILENAME.replace("REMUX", "x265.10bit")
x265_dirname=x265_filename.replace(".mkv","")
x265_fullpath=os.path.join(PROCUCT_DEST_DIR, x265_dirname, x265_filename)

remux_filename=REMUX_DIRNAME+".mkv"
remux_fullpath=os.path.join(PROCUCT_DEST_DIR, REMUX_DIRNAME, REMUX_DIRNAME+".mkv")

cmdcommon_before=f'''source pthvs.sh
export CRF={crf}
export QC={qcomp}
export AQS={aq_s}
export REMUX_FILE={remux_fullpath}
export CROPX={cropx}
export CROPY={cropy}
export ISREMUX=False'''

cmdcommon_after=""
cmd264=""
cmd265=""

cmd264+=f'''
export CODEC=x264
export STREAM_FILE={x264_filename}.264
export PRODUCT_FILE={x264_fullpath}
mkdir -p $(dirname $PRODUCT_FILE)
pthpipe -i $REMUX_FILE -cx $CROPX -cy $CROPY|pth264 -crf $CRF -qc $QC -aqs $AQS -o $STREAM_FILE |& tee $(dirname $PRODUCT_FILE).log
'''

cmd265+=f'''
export CODEC=x265
export STREAM_FILE={x265_filename}.265
export PRODUCT_FILE={x265_fullpath}
mkdir -p $(dirname $PRODUCT_FILE)
pthpipe -i $REMUX_FILE -10 -cx $CROPX -cy $CROPY|pth265 -crf $CRF -qc $QC -aqs $AQS -o $STREAM_FILE |& tee $(dirname $PRODUCT_FILE).log
'''

cmdremux=f'export PRODUCT_FILE={remux_fullpath}\nexport ISREMUX=True\n'

transcodecmd=f"ffmpeg -y -i $STREAM_FILE -c copy $STREAM_FILE.mp4\n"
transcodecmd+=f"export STREAM_FILE=$STREAM_FILE.mp4\n"

if "DTS" in REMUX_DIRNAME:
    transcodecmd+=f"ffmpeg -y -c:a dca -core_only true -i $STREAM_FILE -i $REMUX_FILE -c:v copy \
-max_muxing_queue_size 1024 -bsf:a dca_core -c:a copy -c:s copy -max_muxing_queue_size 1024 \
-map 0 -map 1:a -map 1:s $PRODUCT_FILE\n"

else:
    if "LPCM" in REMUX_DIRNAME:
        acodec="-c:a flac"
    else:
        acodec="-bsf:a dca_core -c:a dts"
    transcodecmd+=f"ffmpeg -y -i $STREAM_FILE -i $REMUX_FILE -c:v copy \
-max_muxing_queue_size 1024 {acodec} -c:s copy -max_muxing_queue_size 1024 \
-map 0 -map 1:a -map 1:s -strict -2 $PRODUCT_FILE \n"
    
transcodecmd+="mkvpropedit $PRODUCT_FILE --add-track-statistics-tags\n"
cmdcommon_after+=get_captures("$(dirname $PRODUCT_FILE).captures","$PRODUCT_FILE")+"\n"
cmdcommon_after+=f"transmission-create -p -o $(dirname $PRODUCT_FILE).torrent -t https://www.pthome.net/announce.php $(dirname $PRODUCT_FILE)"   

cmdcommon_after+=f'''
sed -i "s/\[lib\(x26[[:digit:]]\).*\]/\1 [info]:/g" $(dirname $PRODUCT_FILE).log
python3 <<!
import os, re, subprocess, requests, json, random
from pymediainfo import MediaInfo
from datetime import datetime
from smms import *
from utils import *
import base64, zlib

DOUBAN_ID="{DOUBAN_ID}"
ACKNOWLEDGEMENT="""{ACKNOWLEDGEMENT}"""
SOUNDTRACK="{SOUNDTRACK}"
SRC="{SRC}"
SUBTITLE_GZ={SUBTITLE_GZ}

metainfo=getmetainfo(DOUBAN_ID)
smms = SMMS('hzlclock', 'Hezl1996')
smms.get_api_token()
picbed=upload_folder("$(dirname $PRODUCT_FILE).captures",smms)

with open(f"$(dirname $PRODUCT_FILE).bbcode.base64.txt","w") as f:
    f.write(getbase64json("$PRODUCT_FILE", src=SRC, logname="$(dirname $PRODUCT_FILE).log",\
            picbed=picbed, metainfo=metainfo, is_remux=$ISREMUX,\
            tracks=SOUNDTRACK,gz=SUBTITLE_GZ,ack=ACKNOWLEDGEMENT))
!
'''

# with open(x264_filename+".sh","w") as f:
#     f.write(f"{cmdcommon_before}\n{cmd264}\n{transcodecmd}\n{cmdcommon_after}")
# with open(x265_filename+".sh","w") as f:
#     f.write(f"{cmdcommon_before}\n{cmd265}\n{transcodecmd}\n{cmdcommon_after}")
with open(remux_filename+".sh","w") as f:
    f.write(f"{cmdcommon_before}\n{cmdremux}\n{cmdcommon_after}")
    f.write(f"\n{cmd264}\n{transcodecmd}\n{cmdcommon_after}")
    f.write(f"\n{cmd265}\n{transcodecmd}\n{cmdcommon_after}")
    
# print(f'bash {x264_filename}.sh;\nbash {x265_filename}.sh;\nbash {remux_filename}.sh;')
print(f'bash {remux_filename}.sh;')

#ffmpeg -i dest.mkv -i remux.mkv -map 0:v -map 0:s -map 1:a -c:v copy -c:a dts -strict -2 dtsout.mkv


bash Taipei.Story.1985.1080p.Carlotta.Bluray.AVC.Remux.DTS-HD.MA.1.0-PTH.mkv.sh;
